In [1]:
import pandas as pd
import numpy as np
import json
import os
import glob

import pickle
from pydantic import BaseModel
from openai import OpenAI

from tenacity import (
                        retry,
                        stop_after_attempt,
                        wait_random_exponential
)

from tqdm import tqdm

In [2]:
pd.set_option('display.max_rows', None)

In [3]:
path = '/home/manoelflorencio/cta_for_jd/testbedXS'
os.chdir(path)
print(os.getcwd())

/home/manoelflorencio/cta_for_jd/testbedXS


In [4]:
@retry(wait=wait_random_exponential(min=1,max=60), stop=stop_after_attempt(6))
def execute_prompt(client, system_msg, user_msg):
    completion = client.chat.completions.create(
                                            model="gpt-4o",
                                            messages=[
                                                        {
                                                            "role": "system", 
                                                             "content": f"{system_msg}"
                                                        },
                                                        {
                                                            "role": "user",
                                                            "content": f"{user_msg}"
                                                        }
                                                    ]
                                            )
    return completion

In [5]:
client = OpenAI()

In [6]:
prompts = np.load('Description_test/table_descriptions_test_2025_01_05/Table_JOIN_Prompts/table_JD_with_columns_v3.npy')
prompts.shape

(378, 2)

In [7]:
table_cartesians = pd.read_csv('Description_test/table_descriptions_test_2025_01_05/table_cartesians.csv')
table_cartesians.shape

(378, 2)

In [8]:
def generate_predictions(system_msg, user_msg, client):
    
    result = execute_prompt(client, system_msg, user_msg)
    jd_prediction = result.choices[0].message.content.split('Answer: ')[-1].strip()
    
    return jd_prediction

In [9]:
prompts[0][1]

'Target Table description: The table "statewise-census-data-in-india-1901-2011" contains annual data detailing various socio-economic indicators such as agriculture growth, literacy rates, industry growth, and population metrics across different regions in India including states and specific areas like Andaman & Nicobar Islands and Andhra Pradesh for the years spanning from 1901 to 2011.\n                  Target Table columns descriptions: [\'This column specifies the regularity at which the data was recorded or reported. In this dataset, all data entries have an annual frequency, indicating that they are collected or aggregated on a yearly basis.\'\n \'This column denotes the year in which the data pertains to, ranging from 1951 to 2011, capturing various demographics and economic growth figures for specific years.\'\n \'This column represents the name of the geographical area (state, district, or city) in India for which the data entry is made. It includes names like Gondiya, Nagala

In [10]:
asnwers = []

for i in tqdm(range(len(prompts))):

    system_msg, user_msg = prompts[i][0], prompts[i][1]
    result = generate_predictions(system_msg, user_msg, client)
    asnwers.append(result)

100%|█████████████████████████████████████████████████████████████████████████████████| 378/378 [04:09<00:00,  1.52it/s]


In [11]:
asnwers

['No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'No',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 '

In [12]:
table_cartesians['JOINABLE'] = asnwers

In [13]:
table_cartesians

,LEFT_TABLE,RIGHT_TABLE,JOINABLE
0,statewise-census-data-in-india-1901-2011.csv,road-ahead-current-road-closures.csv,No
1,statewise-census-data-in-india-1901-2011.csv,property-tie-lines.csv,No
2,statewise-census-data-in-india-1901-2011.csv,public-art.csv,No
3,statewise-census-data-in-india-1901-2011.csv,gvrd-sewer-trunk-mains.csv,No
4,statewise-census-data-in-india-1901-2011.csv,SCS_Staff_Salaries_data_30th_June 2010.csv,No
5,statewise-census-data-in-india-1901-2011.csv,schools.csv,No
6,statewise-census-data-in-india-1901-2011.csv,rental-standards-current-issues.csv,No
7,statewise-census-data-in-india-1901-2011.csv,datasets_579296_1047868_authors.csv,No
8,statewise-census-data-in-india-1901-2011.csv,survey_results_schema.csv,No
9,statewise-census-data-in-india-1901-2011.csv,animal-control-inventory-lost-and-found.csv,No


In [14]:
table_cartesians.to_csv('Description_test/table_descriptions_test_2025_01_05/table_cartesians_JD_with_columns_descriptions_v3.csv', index=False)